##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Exibição de dados de imagem no TensorBoard

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/image_summaries"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/image_summaries.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/image_summaries.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tensorboard/image_summaries.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Usando a **API Image Summary do TensorFlow,** você pode facilmente registrar tensores e imagens arbitrárias para visualizá-los no TensorBoard. Isso pode ser extremamente útil para a amostragem e a análise dos seus dados de entrada ou para a [visualização dos pesos das camadas](http://cs231n.github.io/understanding-cnn/) e [tensores gerados](https://hub.packtpub.com/generative-adversarial-networks-using-keras/). Você também pode registrar dados de diagnóstico como imagens que podem ser úteis durante o desenvolvimento do seu modelo.

Neste tutorial, você aprenderá a usar a API Image Summary para visualizar tensores como imagens. Você também aprenderá a converter uma imagem arbitrária em um tensor e visualizá-lo no TensorBoard. Você analisará um exemplo simples, mas real, que usa Resumos de Imagens para ajudar você a entender o desempenho do seu modelo.


## Configuração

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

TensorFlow 2.x selected.


In [4]:
from datetime import datetime
import io
import itertools
from packaging import version

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.2


# Baixe o dataset Fashion-MNIST

Você criará uma rede neural simples para classificar imagens no dataset [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist). Esse dataset consiste em 70.000 imagens em escala de cinza 28 x 28 de produtos de moda de 10 categorias, com 7.000 imagens por categoria.

Primeiro, baixe os dados:

In [5]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

4423680/4422102 [==============================] - 0s 0us/step


## Visualize uma única imagem

Para entender como a API Image Summary funciona, você agora registrará apenas a primeira imagem de treinamento no seu dataset de treinamento no TensorBoard.

Antes de fazer isso, examine o formato dos seus dados de treinamento:

In [6]:
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])

Shape:  (28, 28)
Label:  9 -> Ankle boot


Observe que o formato de cada imagem no dataset é um tensor de posto 2 de formato (28, 28), representando a altura e a largura.

No entanto, `tf.summary.image()` espera um tensor de posto 4 com `(batch_size, height, width, channels)`. Portanto, os tensores precisam ser remodelados.

Você está registrando só uma imagem, então o `batch_size` é 1. As imagens são em escala cinza, então defina `channels` como 1.

In [ ]:
# Reshape the image for the Summary API.
img = np.reshape(train_images[0], (-1, 28, 28, 1))

Você já está pronto para registrar essa imagem e visualizá-la no TensorBoard.

In [ ]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
  tf.summary.image("Training data", img, step=0)

Agora, use o TensorBoard para examinar a imagem. Aguarde alguns segundos para a interface do usuário inicializar.

In [ ]:
%tensorboard --logdir logs/train_data

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_single.png?raw=1"/> -->

O painel de controle "Time Series" (Série temporal) mostra a imagem que você acabou de registrar. É uma "ankle boot" (bota de cano curto).

A imagem é redimensionada em um tamanho padrão para facilitar a visualização. Se você quiser ver a imagem original não dimensionada, marque "Show actual image size" (Mostrar tamanho real da imagem) na parte inferior do painel "Settings" à direita.

Ajuste os controles deslizantes de brilho e contraste para ver como eles afetam os pixels da imagem.

## Visualize várias imagens

Registrar um tensor é ótimo, mas e se você quiser registrar vários exemplos de treinamento?

Basta especificar o número de imagens que você quer registrar ao passar dados para `tf.summary.image()`.

In [ ]:
with file_writer.as_default():
  # Don't forget to reshape.
  images = np.reshape(train_images[0:25], (-1, 28, 28, 1))
  tf.summary.image("25 training data examples", images, max_outputs=25, step=0)

%tensorboard --logdir logs/train_data

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_multiple.png?raw=1"/> -->

## Registre dados de imagens arbitrárias

E se você quiser visualizar uma imagem que não é um tensor, como uma imagem gerada por [matplotlib](https://matplotlib.org/)?

Você precisa de algum código boilerplate para converter a plotagem em um tensor, mas, depois disso, estará tudo pronto.

No código abaixo, você registrará as primeiras 25 imagens como uma bela grade usando a função `subplot()` de matplotlib. Em seguida, você visualizará a grade no TensorBoard:

In [ ]:
# Clear out prior logging data.
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
  
  return figure

# Prepare the plot
figure = image_grid()
# Convert to image and log
with file_writer.as_default():
  tf.summary.image("Training data", plot_to_image(figure), step=0)

%tensorboard --logdir logs/plots

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_arbitrary.png?raw=1"/> -->

## Crie um classificador de imagens

Agora, combine tudo isso em um exemplo real. Afinal, você está aqui para o aprendizado de máquina, e não para plotar imagens bonitas!

Use os resumos de imagens para entender o desempenho do seu modelo ao treinar um classificador simples para o dataset Fashion-MNIST.

Primeiro, crie um modelo bastante simples e compile-o, configurando o otimizador e a função de perda. O passo de compilação também especifica que você quer registrar a exatidão do classificador ao longo do processo.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Ao treinar um classificador, é útil conferir a [matriz de confusão](https://en.wikipedia.org/wiki/Confusion_matrix). Ela fornece informações detalhadas sobre o desempenho do classificador com dados de teste.

Defina uma função para calcular a matriz de confusão. Use uma função [Scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html) conveniente para isso e depois faça a plotagem usando matplotlib.

In [ ]:
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

Você já está pronto para treinar o classificador e registrar regularmente a matriz de confusão ao longo do processo.

Faça o seguinte:

1. Crie a [callback do TensorBoard para o Keras](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) para registrar métricas básicas.
2. Crie uma [LambdaCallback do Keras](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LambdaCallback) para registrar a matriz de confusão no final de cada época.
3. Treine o modelo usando Model.fit(), garantindo que ambas as callbacks sejam passadas.

Conforme o treinamento avançar, role para baixo para ver o TensorBoard inicializar.

In [ ]:
# Clear out prior logging data.
!rm -rf logs/image

logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("epoch_confusion_matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Start TensorBoard.
%tensorboard --logdir logs/image

# Train the classifier.
model.fit(
    train_images,
    train_labels,
    epochs=5,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_images, test_labels),
)

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_accuracy.png?raw=1"/> -->

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_cm.png?raw=1"/> -->

Observe que a exatidão está aumentando em ambos os datasets de treinamento e validação. Isso é um bom sinal. Mas como é o desempenho do modelo em subsets de dados específicos?

Role para baixo no painel de controle "Time Series" para visualizar as matrizes de confusão registradas. Marque "Show actual image size" na parte inferior do painel "Settings" para ver a matriz de confusão em tamanho real.

Por padrão, o painel de controle mostra o resumo de imagem para o último passo ou época registrada. Use o controle deslizante para visualizar as matrizes de confusão anteriores. Observe como a matriz muda significativamente à medida que o treinamento avança, com os quadrados mais escuros se unindo na diagonal e o resto da matriz tendendo a 0 e branco. Isso significa que seu classificador está melhorando com o avanço do treinamento. Bom trabalho!

A matriz de confusão mostra que o modelo simples tem alguns problemas. Apesar do ótimo progresso, camisas, camisetas e suéteres estão se confundindo uns com os outros. O modelo precisa ser melhorado.

Se você tiver interesse, tente melhorar esse modelo com uma [rede convolucional](https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a) (CNN).